<a href="https://colab.research.google.com/github/arunm917/Climate-Action-Tool/blob/main/solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading packages and files

In [22]:
import numpy as np
import pandas as pd
import gdown
import regex as re
from scipy.optimize import minimize

In [23]:
# from google.colab import files
# uploaded = files.upload()

In [24]:
# downloading file from gdrive
output = 'equations'
file_id = '1q9cRaPAXvg2KSuc9f8WDjnfLQDFdmtew' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1q9cRaPAXvg2KSuc9f8WDjnfLQDFdmtew
To: /content/equations
100%|██████████| 578/578 [00:00<00:00, 428kB/s]


DONE.


In [25]:
# downloading file from gdrive
output = 'parameters'
file_id = '1qC5eDYSwAqwYZ0QqlVpayiuQnix9e85K' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qC5eDYSwAqwYZ0QqlVpayiuQnix9e85K
To: /content/parameters
100%|██████████| 208/208 [00:00<00:00, 522kB/s]


DONE.


In [26]:
# downloading file from gdrive
output = 'variables'
file_id = '1qAyqEmDgX0hvR2LN3moC6i3SDNrlylYZ' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qAyqEmDgX0hvR2LN3moC6i3SDNrlylYZ
To: /content/variables
100%|██████████| 218/218 [00:00<00:00, 659kB/s]


DONE.


# Processing files

In [27]:
parameters = pd.read_csv('parameters')

In [28]:
parameters

,parameters,values
0,phi_crusher,3.50
1,phi_rm,26.00
2,phi_cm,76.30
3,phi_phk,37.70
4,phi_cooler,37.70
5,phi_cementmill,30.00
6,w_add_ls,0.11
7,w_c_ls,0.06
8,w_c_coal,1.00
9,w_cl_rm,0.64


In [29]:
variables_df = pd.read_csv('variables')
variables_df

,variables,values
0,m_ls,0
1,m_rm,0
2,m_rawmeal,0
3,m_pcoal,0
4,m_phk_air,0
5,m_hotclinker,0
6,m_stackgas,0
7,m_clinker,0
8,m_cement,0
9,m_co2,0


In [30]:
variables_list = list(variables_df['variables'])

In [31]:
print(variables_list)

['m_ls', 'm_rm', 'm_rawmeal', 'm_pcoal', 'm_phk_air', 'm_hotclinker', 'm_stackgas', 'm_clinker', 'm_cement', 'm_co2', 'e_crusher', 'e_rawmill', 'e_coalmill', 'e_phk_total', 'e_cooler', 'e_cementmill']


In [32]:
with open('equations', 'r') as f:
    # Read the equations line by line
    eq_lines = f.readlines()

In [33]:
print(eq_lines)

['eq1: w_add_ls * m_ls - m_rm + 100\n', 'eq2: m_rm - m_rawmeal\n', 'eq3: (w_c_ls/w_c_coal) * m_ls - m_pcoal\n', 'eq4: (1 - w_cl_rm) * m_rawmeal + w_c_coal * m_pcoal + m_phk_air - m_stackgas\n', 'eq5: m_hotclinker + w_air_ls * m_ls - m_phk_air - m_clinker\n', 'eq6: m_clinker + w_gy_ls * m_ls + w_fly_ls * m_ls - m_cement\n', 'eq7: e_crusher - phi_crusher * m_rm\n', 'eq8: e_rawmill - phi_rm * m_rawmeal\n', 'eq9: e_coalmill - phi_cm * m_pcoal\n', 'eq10: e_phk_total - phi_phk * m_hotclinker - (w_c_ls * 393.5/12) * m_pcoal\n', 'eq11: e_cooler - phi_cooler * m_hotclinker\n', 'eq12: e_cementmill - phi_cementmill * m_cement\n']


In [34]:
# Create a list to store the equations
eq_list = []

# Loop through the equation lines
for eq_line in eq_lines:
    # Split the line into the equation name and the equation expression
    eq_name, eq_expr = eq_line.strip().split(':')
    # Convert the tuple of symbols to a single expression
    eq_list.append(eq_expr) 
    # Add the equation to the dictionary
print(eq_list)

[' w_add_ls * m_ls - m_rm + 100', ' m_rm - m_rawmeal', ' (w_c_ls/w_c_coal) * m_ls - m_pcoal', ' (1 - w_cl_rm) * m_rawmeal + w_c_coal * m_pcoal + m_phk_air - m_stackgas', ' m_hotclinker + w_air_ls * m_ls - m_phk_air - m_clinker', ' m_clinker + w_gy_ls * m_ls + w_fly_ls * m_ls - m_cement', ' e_crusher - phi_crusher * m_rm', ' e_rawmill - phi_rm * m_rawmeal', ' e_coalmill - phi_cm * m_pcoal', ' e_phk_total - phi_phk * m_hotclinker - (w_c_ls * 393.5/12) * m_pcoal', ' e_cooler - phi_cooler * m_hotclinker', ' e_cementmill - phi_cementmill * m_cement']


In [35]:
# Creating dictionary for parameters and values
param_dict = dict(zip(parameters['parameters'],parameters['values']))
print(param_dict)

{'phi_crusher': 3.5, 'phi_rm': 26.0, 'phi_cm': 76.3, 'phi_phk': 37.7, 'phi_cooler': 37.7, 'phi_cementmill': 30.0, 'w_add_ls': 0.11, 'w_c_ls': 0.06, 'w_c_coal': 1.0, 'w_cl_rm': 0.64, 'w_air_ls': 1.64, 'w_gy_ls': 0.05, 'w_fly_ls': 0.33}


In [36]:
# Substituting parameters in equation with their values
modified_list = []
for eq in eq_list:
  for key in param_dict:
    if key in eq:
        value = param_dict.get(key)
        eq = eq.replace(key, str(param_dict.get(key)))
  modified_list.append(eq)
print(modified_list)

[' 0.11 * m_ls - m_rm + 100', ' m_rm - m_rawmeal', ' (0.06/1.0) * m_ls - m_pcoal', ' (1 - 0.64) * m_rawmeal + 1.0 * m_pcoal + m_phk_air - m_stackgas', ' m_hotclinker + 1.64 * m_ls - m_phk_air - m_clinker', ' m_clinker + 0.05 * m_ls + 0.33 * m_ls - m_cement', ' e_crusher - 3.5 * m_rm', ' e_rawmill - 26.0 * m_rawmeal', ' e_coalmill - 76.3 * m_pcoal', ' e_phk_total - 37.7 * m_hotclinker - (0.06 * 393.5/12) * m_pcoal', ' e_cooler - 37.7 * m_hotclinker', ' e_cementmill - 30.0 * m_cement']


In [37]:
for item in modified_list:
  print(item)

 0.11 * m_ls - m_rm + 100
 m_rm - m_rawmeal
 (0.06/1.0) * m_ls - m_pcoal
 (1 - 0.64) * m_rawmeal + 1.0 * m_pcoal + m_phk_air - m_stackgas
 m_hotclinker + 1.64 * m_ls - m_phk_air - m_clinker
 m_clinker + 0.05 * m_ls + 0.33 * m_ls - m_cement
 e_crusher - 3.5 * m_rm
 e_rawmill - 26.0 * m_rawmeal
 e_coalmill - 76.3 * m_pcoal
 e_phk_total - 37.7 * m_hotclinker - (0.06 * 393.5/12) * m_pcoal
 e_cooler - 37.7 * m_hotclinker
 e_cementmill - 30.0 * m_cement


# Evaluating coefficients

In [38]:
corrected_equations = []
expressions_inside_parentheses = []
for equation in modified_list:
  matches = re.findall(r'(\(.*?\))', equation)
  # print(matches)
  for expression in matches:
    # print(expression)
    eval_coeff = eval(expression)
    # print(eval_coeff)
    equation = equation.replace(expression, str(eval_coeff))
  # expressions_inside_parentheses.extend(matches)
  corrected_equations.append(equation)

print(corrected_equations)
# print(expressions_inside_parentheses)

[' 0.11 * m_ls - m_rm + 100', ' m_rm - m_rawmeal', ' 0.06 * m_ls - m_pcoal', ' 0.36 * m_rawmeal + 1.0 * m_pcoal + m_phk_air - m_stackgas', ' m_hotclinker + 1.64 * m_ls - m_phk_air - m_clinker', ' m_clinker + 0.05 * m_ls + 0.33 * m_ls - m_cement', ' e_crusher - 3.5 * m_rm', ' e_rawmill - 26.0 * m_rawmeal', ' e_coalmill - 76.3 * m_pcoal', ' e_phk_total - 37.7 * m_hotclinker - 1.9675 * m_pcoal', ' e_cooler - 37.7 * m_hotclinker', ' e_cementmill - 30.0 * m_cement']


In [39]:
for item in corrected_equations:
  print(item)

 0.11 * m_ls - m_rm + 100
 m_rm - m_rawmeal
 0.06 * m_ls - m_pcoal
 0.36 * m_rawmeal + 1.0 * m_pcoal + m_phk_air - m_stackgas
 m_hotclinker + 1.64 * m_ls - m_phk_air - m_clinker
 m_clinker + 0.05 * m_ls + 0.33 * m_ls - m_cement
 e_crusher - 3.5 * m_rm
 e_rawmill - 26.0 * m_rawmeal
 e_coalmill - 76.3 * m_pcoal
 e_phk_total - 37.7 * m_hotclinker - 1.9675 * m_pcoal
 e_cooler - 37.7 * m_hotclinker
 e_cementmill - 30.0 * m_cement


# Solving the optimization problem

In [51]:
# Define the objective function
def objective_function(decision_variables):
    # x, y, z, w = initial_guess
    for index, variable in enumerate(variables_list):
      globals()[variable] = decision_variables[index]

    equations = []
    for equation in corrected_equations:
      equations.append(eval(equation))
    squared_errors = [result**2 for result in equations]
    return sum(squared_errors)

In [52]:
def constraints(decision_variables):

    for index, variable in enumerate(variables_list):
      globals()[variable] = decision_variables[index]

    constraints = []
    for equation in corrected_equations:
      constraints.append(eval(equation))
    return constraints

In [57]:
# Initial guess for the decision variables
initial_guess = [1] * len(variables_list)

# Bounds for the decision variables
bounds = [(0, 1000)]* len(variables_list)

result = minimize(objective_function, initial_guess, bounds = bounds, constraints={'type': 'eq', 'fun': constraints})

In [58]:
# Extract the optimal solution
optimal_solution = result.x
print("Optimal solution:", optimal_solution)

Optimal solution: [4.81031876e-01 3.90533399e+01 3.90533399e+01 6.07406955e-01
 3.22653345e-01 6.21628766e-01 1.41522171e+01 6.93964687e-01
 6.42876888e-01 1.00000000e+00 1.35148006e+02 1.00000000e+03
 3.49867111e-14 8.31658074e-01 8.47528864e-01 1.43757661e+00]
